In [1]:
import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("dataOneHot/", one_hot=True)

W0806 13:27:56.134912 140369879922432 deprecation.py:323] From <ipython-input-1-e5ed91b8b8cc>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0806 13:27:56.135572 140369879922432 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0806 13:27:56.136625 140369879922432 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting dataOneHot/train-images-idx3-ubyte.gz
Extracting dataOneHot/train-labels-idx1-ubyte.gz
Extracting dataOneHot/t10k-images-idx3-ubyte.gz


W0806 13:27:56.363155 140369879922432 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting dataOneHot/t10k-labels-idx1-ubyte.gz


In [2]:
# 입력값과 출력값을 받기 위한 플레이스홀데를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [3]:
# 변수들을 설정하고 소프트맥스 회귀 모델을 정의합니다.
W = tf.Variable(tf.zeros(shape=[784,10]), name='W')
b = tf.Variable(tf.zeros(shape=[10]), name='b')
logits = tf.matmul(x, W) + b
y_pred = tf.nn.softmax(logits)

In [4]:
# cross-engropy 손실 함수와 옵티마이저를 정의합니다.
#loss_function = -tf.reduce_sum(y*tf.log(y_pred), reduction_indices=[1])
loss_function = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y)
loss = tf.reduce_mean(loss_function)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 텐서보드를 위한 요약 정보(scalar)를 정의합니다.
tf.summary.scalar("Loss", loss)

<tf.Tensor 'Loss:0' shape=() dtype=string>

In [5]:
# 세션을 열고 변수들에 초기값을 할당합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 텐서보드 요약정보들을 하나로 합칩니다.
merged = tf.summary.merge_all()
# 텐서보드 summary 정보들을 저장할 폴더 경로를 설정합니다.
tensorboard_writer = tf.summary.FileWriter('./tensorboard_log', sess.graph)

In [6]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y:batch_ys})
    summary = sess.run(merged, feed_dict={x: batch_xs, y: batch_ys})
    tensorboard_writer.add_summary(summary, i)
    if i%1000==999:
        print("#",i+1,": ", sess.run(loss_function, feed_dict={x:batch_xs, y:batch_ys}))

# 1000 :  [4.83858585e-02 1.11566754e-02 1.62467241e+00 1.27039135e-01
 8.09971020e-02 2.59063556e-03 6.79882569e-03 2.04964489e-01
 1.65269412e-02 7.91957881e-03 3.49928886e-02 8.35131288e-01
 1.20671779e-01 1.35913468e-03 3.05222115e-03 9.35657695e-03
 1.10415317e-01 2.67820917e-02 2.37563951e-03 5.61652966e-02
 2.10642610e-02 1.96775854e-01 1.98658332e-02 1.05365179e-02
 1.52709649e-03 6.16476722e-02 1.04905240e-01 1.67515548e-03
 2.03307763e-01 2.10657090e-01 7.92831853e-02 1.14985496e-01
 5.92452634e-05 6.37445450e-02 1.03038418e+00 1.14907287e-01
 1.66380946e-02 1.85353530e-03 7.17032924e-02 3.05171125e-05
 3.71551816e-03 1.26586735e-01 1.28725767e-02 6.38730526e-02
 1.06543526e-02 1.05325517e-03 6.14772923e-02 4.31575961e-02
 1.64476126e-01 1.06514040e-02 3.52639735e-01 4.78432653e-03
 1.41916975e-01 3.55297744e-01 8.19829479e-02 4.29994822e-01
 1.04769459e-02 1.36471121e-02 4.36689347e-01 4.67523098e-01
 1.20670688e+00 9.31949355e-03 1.22068596e+00 3.44849401e-03
 1.14978206e+0

In [7]:
# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('정확도: %f' %sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels}))

정확도: 0.921300


In [8]:
sess.close()